In [1]:
# import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import shap
import matplotlib.pyplot as plt

/Users/jasongrewal/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


In [2]:
seed = 2724

### Import data

In [3]:
DF_PATH = "mod05_data/sample.csv"
df = pd.read_csv(DF_PATH)

FileNotFoundError: [Errno 2] No such file or directory: 'mod05_data/sample.csv'

### Separate data by independent (X) and dependent (y) variables

In [ ]:
X = df[["income", "education_years", "zipcode_score"]]
y = df["target"]

### Split the data into a _training_ set (to build a model) and _test_ set (to validate a model)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=seed
)

### Build a model on the training set

In [ ]:
model = RandomForestRegressor(
    n_estimators=200,
    random_state=seed
)
model.fit(X_train, y_train)

### Use SHAP to explain the model on test data

In [ ]:
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

This will allow us to see which variables are most important to predicting the outcome.

In [ ]:
shap.plots.bar(shap_values)

### Import the `group` variable, which was **not** used in training this model.

In [ ]:
X_test_with_group = X_test.copy()
X_test_with_group["group"] = df.loc[X_test.index, "group"]

### Look at the difference in SHAP values between the two groups across the variables used in the model.

In [ ]:
shap_df = pd.DataFrame(shap_values.values, columns=X_test.columns)
shap_df["group"] = X_test_with_group["group"].values

shap_df.groupby("group").mean()

### Let's put `group` and `zipcode_score` in the same plot

In [ ]:
def plot_shap(var):
    # Extract SHAP values for the feature
    shap_var = shap_values[:, var].values

    # Plot the values of each group using different colors
    plt.figure()
    plt.scatter(
        X_test[var],
        shap_var,
        c=X_test_with_group["group"]
    )
    plt.xlabel(var)
    plt.ylabel(f"SHAP value for var")
    plt.title("Proxy feature impact by group")
    plt.show()

plot_shap("zipcode_score")

# Discussion Questions

### What is a _SHAP_ (or Shapley) value? 

It’s basically a way to assign credit to each feature for a model's prediction. Think of it like a group project where you calculate exactly how much each person or variable contributed to the final grade. It helps you see which factors are actually driving the model's decisions.

### Suppose you built this model and then it is peer reviewed by another entity. If the reviewer asks whether you used the variable `group` in your model, what would your answer be?

I’d tell them no because we usually leave out protected labels like race or gender to avoid direct bias. The goal is to make sure the model isn't explicitly making decisions based on someone's identity. It’s a standard first step in trying to build a blind model.

### If the reviewer asks whether the outcome of your model is correlated with `group`, what would your answer be?

I’d have to say yes, because just deleting a label doesn't magically fix the bias in the rest of the data. Other variables act as stand ins for that group, so the model still ends up picking up on those demographic patterns indirectly. It’s a classic case of "what you don't know can still hurt you.

### Construct a "proxy feature impact by group" plot for `income`. How is this plot different from the one for `zipcode_score`?

In [ ]:
plot_shap("income"). The income plot would likely show that more money helps everyone's score fairly consistently across the board. However, the zipcode_score plot would probably show a much uglier split because zip codes are often tied to where specific groups live. This makes zipcode_score a much "louder" proxy for bias than income ever would be.

Type your answer here.

### If, instead, you were the **reviewer**, what other questions might you ask the person who built this model? Give at least two.

How do you plan to handle the bias found in those proxy variables? Which specific fairness metrics are you using to measure overall success?